In [45]:
# Douglas 
# 6/11/2019

In [14]:
import pandas as pd 
pd.set_option('max_colwidth', -1)

In [3]:
!ls data/

disaster-relief-dfe-854578.csv		  paired_terms_disaster_relef.csv
Emergency_Help_Requests_from_Twitter.csv  socialmedia-disaster-tweets-DFE.csv


In [7]:
help_reqs = pd.read_csv('./data/Emergency_Help_Requests_from_Twitter.csv')
# Data source: http://localhost:8888/?token=bb82409f3904772fa6e5e907c013a527b8b62aa7c803b879

In [9]:
help_reqs.shape

(85, 20)

In [10]:
help_reqs.columns

Index(['X', 'Y', 'OBJECTID', 'displayName', 'username', 'latitude',
       'longitude', 'message', 'imageUrl', 'type', 'subType', 'dcId',
       'locationType', 'locationDetails', 'algorithmType', 'algorithmMatch',
       'date', 'postUrl', 'profileUrl', 'replyUrl'],
      dtype='object')

In [15]:
help_reqs['message']

0     @mastasj 2231 Fredrick st port Arthur to 776404097208322 Linda chambers                                                                     
1     RESCUE ALERT: Six people in Port Arthur need rescuing. They're at 5235 Ebony Lane with water knee deep into their home.                     
2     2320 COX ST PORT ARTHUR TX 77640 !!!!!!!!!!                                                                                                 
3     .@HarveyRescue https://t.co/HqJcQPkLJH                                                                                                      
4     @austin_milan y'all got a team out in PA? https://t.co/6zptz4mnKG                                                                           
5     Need a boat rescue at 2611 east 12th street, Port Arthur, 77640.  Donald Sennette (74 y/o old).… https://t.co/yUjopFdo6L                    
6     If it's not one thing . It's another! My great grandma , Ophelia Kinlaw is in need of rescue! 3401 10 st Port Ar

In [ ]:
# To do:


In [40]:
%%time 
harvey = pd.read_csv('./data/HurricaneHarvey_ids.txt.gz', header=None, names=['id'])['id'].tolist()
# https://digital.library.unt.edu/ark:/67531/metadc993940/

CPU times: user 4.14 s, sys: 179 ms, total: 4.32 s
Wall time: 4.29 s


Keywords used to generate dataset
    #Harvey
    #Harvey2017
    #HarveyStorm
    #HoustonFlood
    #HoustonFlooding
    #HoustonFloods
    #HurricaneHarvey
    Gulf Coast
    Hurricane Harvey
    Twitter

In [36]:
# 7 million tweets 
harvey.shape

(7041866, 1)

In [23]:
from twarc import Twarc

In [28]:
import gzip

In [44]:
# %%time
# with gzip.open('./data/HurricaneHarvey_ids.txt.gz', 'rt') as f: #rt=read text
#     tweet_ids = [line[:-1] for line in f] # remove \n char
# pandas does it faster and easier
# https://stackoverflow.com/questions/10566558/python-read-lines-from-compressed-text-files

CPU times: user 5.45 s, sys: 285 ms, total: 5.74 s
Wall time: 5.77 s


In [42]:
# tweet_ids[:5]

['898676461032189952',
 '898679833554759680',
 '898680216989696000',
 '898681045884715008',
 '898681292643909632']

In [48]:
# https://github.com/DocNow/twarc
# Create instance - filling in crednetials from .twarc file
t = Twarc()

In [49]:
tweets = []
for tweet in t.hydrate(harvey[:5]):
    tweets.append(tweet)

In [52]:
tweets[0]

{'contributors': None,
 'coordinates': None,
 'created_at': 'Fri Aug 18 22:55:43 +0000 2017',
 'display_text_range': [0, 120],
 'entities': {'hashtags': [{'indices': [32, 39], 'text': 'Harvey'},
   {'indices': [64, 73], 'text': 'Coahuila'},
   {'indices': [74, 84], 'text': 'NuevoLeon'},
   {'indices': [85, 96], 'text': 'Tamaulipas'}],
  'media': [{'display_url': 'pic.twitter.com/DGW2NdvgpD',
    'expanded_url': 'https://twitter.com/mmsoriano/status/898639604240203776/photo/1',
    'id': 898639595323047936,
    'id_str': '898639595323047936',
    'indices': [97, 120],
    'media_url': 'http://pbs.twimg.com/media/DHib82RUwAAjE9i.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/DHib82RUwAAjE9i.jpg',
    'sizes': {'large': {'h': 736, 'resize': 'fit', 'w': 897},
     'medium': {'h': 736, 'resize': 'fit', 'w': 897},
     'small': {'h': 558, 'resize': 'fit', 'w': 680},
     'thumb': {'h': 150, 'resize': 'crop', 'w': 150}},
    'source_status_id': 898639604240203776,
    'source_statu

In [ ]:
tweets[0]['full_text'] # extract text